In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
## some modules are imported later

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.base import is_classifier

logr = LogisticRegression()
type(logr)
is_classifier(logr)


# Looking at the Data

In [ ]:
housing = pd.read_csv("../datasets/housing/housing.csv")
housing.head()

In [ ]:
## Let's see how many columns and rows we have. This also highlights the number of null values which may need to be dealt with.
housing.info()

In [ ]:
## All of the columns appear to be numeric, apart from the last one, ocean_proximiity, which is an object. Let's explore what that is.
housing["ocean_proximity"].value_counts()
## Looks like it is catagorical, with levels, INLAND, NEAR OCEAN, NEAR BAY and ISLAND.

In [ ]:
## Let's look at the numerical summary
housing.describe()

In [ ]:
%matplotlib inline
## Let's visualise the distributions of the numeric variables
housing.hist(bins = 50, figsize = (20, 15))
plt.show()

# Splitting the Data into Testing and Training Sets

In [ ]:
## if out groups are equal, then we can simply split our data using the train_test_split() function.
## providing a random state ensures consistency if ran many times - this function is extracting rows from a index randomly generated, 
## so the same rows are removed every time we run this

train_set, test_set = train_test_split(housing, test_size = .2, random_state = 42)

In [ ]:
## However when we make income categorical, this method of splitting the data will introduce sampling bias, as not all groups are equal.
## One useful solution is to use stratified sampling, to ensure the train and test data have the same proportions of the groups.

housing["income_cat"] = pd.cut(housing["median_income"],                ## what column do you want to split into bins?
                              bins = [0., 1.5, 3.0, 4.5, 6., np.inf],  ## what do the bins look like?
                              labels = [1, 2, 3, 4, 5])                ## what are the names of these bins?

split = StratifiedShuffleSplit(n_splits = 1, test_size = .2, random_state = 42)
## the for loop is not really iterating, just extracting the values and mapping them
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
for i1, i2 in split.split(housing, housing["income_cat"]):
    print(i1, i2)

In [ ]:
## let's double check the proportions, to make sure it worked
strat_train_set["income_cat"].value_counts() / len(strat_train_set)

In [ ]:
## let's double check the proportions, to make sure it worked
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
## we don't need to convert the income variable into a factor, but this is useful to be aware of
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis = 1, inplace = True)

# Visualising the Data

In [ ]:
housing = strat_train_set.copy()
## Let's see what areas are more density populated in our sample by using a scatterplot with transparency.
housing.plot(kind = "scatter", x = "longitude", y = "latitude", alpha = .1)

In [ ]:
## Let's try to recrete this graphic but adding in details about the house values, to see if there are any obvious patterns.
housing.plot(kind = "scatter",                   ## Type of graph
             x = "longitude",                    ## What goes on the x axis
             y = "latitude",                     ## What goes on the y axis
             alpha = .4,                         ## The transparency to use
             s = housing["population"]/100,      ## How to side the points - i.e by population
             label = "population",               ## The name of the legend
             figsize = (10, 7),                  ## The size of the graph
             c = "median_house_value",           ## How to colour the points - i.e by prize
             cmap = plt.get_cmap("jet"),         ## What palette to use, plt.get_cmap()
             colorbar = True                     ## Should we show the colourbar?
            )
plt.legend()

So we can now see that the areas near the ocean are typically higher in value. Also smaller areas tend to be cheaper too, likely less influenced by the demand that comes from higher population density.

Next, let's check for correlation between all of our variables, since this dataset is relatively small.

In [ ]:
# corr_matrix = housing.corr(method = "spearman", numeric_only = True)
# corr_matrix["median_house_value"].sort_values(ascending = False)

In [ ]:
corr_matrix = housing.corr(method = "pearson", numeric_only = True)
corr_matrix["median_house_value"].sort_values(ascending = False)

Looks like we have mostly weak correlations for `median_house_value`, but `median_income` is moderately correlated with a value of 0.687. Surprising, total_rooms only has a correlation of 0.13. Maybe having room size, number of bedrooms, number of bathrooms or total area would be better.

Another way to see correlation is to use the `pandas.plotting module`, to get the function `scatter_matrix()`. We set the diagonal to *"kde"*, which means the kernel density estimate. Again, the data does not appear to be normally distributed, and the limits on the data are becoming more clear. Especially for our target variable, the limit of 500,000 generates an almost solid line. There are also lines around 350,000 and 450,000.

In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]

pd.plotting.scatter_matrix(
    housing[attributes],
    figsize = (12, 9), 
    diagonal = "kde", 
    alpha = .3
    )
plt.show()

Some of our variables could be improved, for example *households* and *total_bedrooms* could be combined to get number of bedroom in a household. 

In [ ]:
housing["rooms_per_household"] = housing["total_bedrooms"] / housing["households"]
housing["bedroom_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]

In [ ]:
corr_matrix = housing.corr(method = "pearson", numeric_only = True)
corr_matrix["median_house_value"].sort_values(ascending = False)

# Preparing for Machine Learning Algorithms

We are going to revert back to the original traning data, but we can come back to do some feature engineering and generate better variables.

## Categorical Feature

In [ ]:
## .drop() creats a copy first so we still have the median_house_value column, and that's why the second line works too.
housing = strat_train_set.drop("median_house_value", axis = 1)
housing_labels = strat_train_set["median_house_value"].copy()

housing.info() ## 16,512 rows

The first thing we need to deal with is the missing values, we have a few options:
- delete the column(s) containing missing values
- delete the row(s) containing missing values
- fill the missing values for another value (Such as the mean, median, zero, etc...)


In [ ]:
### Removes the entire column
# housing.drop("total_bedrooms", axos = 1)
### Removed the rows
# housing.dropna(subset=["total_bedrooms"])
### Fills in the missing values
median = housing["total_bedrooms"].median()
# housing["total_bedrooms"].fillna(median, inplace = True)  ## Depreciated syntax, replace more explicitly.
housing["total_housing"] = housing["total_bedrooms"].fillna(median)

One not so obvious thing to remember for the last option, is that this median value needs to be applied globally. So it needs to be available to the testing data as well as new data, if we are training incrementally.
This could become problematic to keep tract of, but `Scikit-learn` has a useful class called `SimpleImputer`. This is for univariate values, as the multivariate alternative is the `IterativeImputer`. There is also the K-nearest neighbours imputer, likely for categorical missing values:`KNNImputer`

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = "median") ## other strategies include: mean, constant and most-frequent (mode)

To use this imputer, we need to fit it to our data but it only works for numerical values. First we need to remove the non-numerical values, i.e. the `ocean_proximity` category. Then we can use the `SimpleImputer`'s method `fit()` to calculate the median of all the columns.

In [ ]:
housing_num = housing.drop("ocean_proximity", axis = 1)

## calculate the median for each of the columns
imputer.fit(housing_num)

## to see the imputer values, we can use the attribute statistics_
imputer.statistics_

In [ ]:
## now transform the original dataframe
X = imputer.transform(housing_num)
X

In [ ]:
## To convert this numpy array back into a pandas dataframe, we can:
housing_tr  = pd.DataFrame(X, columns = housing_num.columns, index = housing_num.index)

## alternatively, we could use the convenient, fit_transform() method, although I don't know if this outputs a Panda's Dataframe
housing_tr.info()

Next we need to handle the categorical variable. Machine learning algorithms are numerical, we could convert the categories into a dummy variable, like what R does for linear models. 

In [ ]:
housing_cat = housing[["ocean_proximity"]] ## double [[ ]] means it returns a pandas dataframe instead of a series.
housing_cat.head()

We could also use an ordinal encoder, it does make the assumption that the ordinal values are equally spaced. 

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

Maybe a better solution is **one-hot encoding**, where we create a vector containing zero's in all places apart from one, the category it represents.

If we had three groups, it would look like:
- Group 1 = [1, 0, 0]
- Group 2 = [0, 1, 0]
- Group 3 = [0, 0, 1]

In general, the length of the vector is the number of groups, and we create a *sparse* vector to represent the categories. The Sparse vector comes from SciPy, that is where the only values stored represents the index of the non-zero values. So if we have a vector of length 20, and the 13th element is a 1, then the sparse matrix/vector would store 13. This prevents the waste of memory to store lots of zeros.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()

housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

We can convert this to a *dense* matrix is we want, by using the `toarray()` method, which returns a NumPy array/matrix.

In [ ]:
housing_cat_1hot.toarray()

This method is not perfect though, if we have a lot of categories, this could negatively impact training speeds. One potential and common solution is to use *representation learning*, where a numerical value associated with only one category is given, this could be the population or the GDP of the country. This is called embedding, as we are embedding a low-dimensional feature to represent a categorical feature into the data, so that the machine can quickly learn to differentiate between the levels of the category.

## Defining our own Transformers

Defiining our own transformers can help automate the feature enginerring so that we can test more parameters and save time.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

## column index for the matrix/dataframe
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CominedAttributesAdder(BaseEstimator, TransformerMixin):
    ## we use BaseEstimate to avoid using *args and **kwargs since it has get_params() and set_params() methods
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y = None):
        return self ## that's it

    ## we are only transforming the features, so we don't need y.
    def transform(self, X):
        ## define the new panda's series containing the engineered features.
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]

        ## check if we also want bedrooms per room
        if self.add_bedrooms_per_room:
            
            bedrooms_per_room =  X[:, bedrooms_ix] / X[:, household_ix]
            ## add to original dataframe
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
            
        else:
            ## add to original dataframe
            return np.c_[X, rooms_per_household, population_per_household]
            
        

In [ ]:
attr_adder = CominedAttributesAdder(add_bedrooms_per_room = False)
housing_extra_attribs= attr_adder.transform(housing.values)  ## values attribute returns a numpy array.
housing_extra_attribs ## returns the updated numpy array.

## Feature Scaling 

The two main types of scaling are *normalisation* and *standardisation*. Normalisation (also called Min-Max scaling) bounds the feature to 0 and 1 by subtracting each value by the minimum value, then dividing by the different of the maximum and minimun value. That mean if the minimun value was 5, and the maximum was 10, the value 5 becomes 0 / 5 and 10 becomes 5 / 5. 

Standardisation is when we find the difference between the value and the mean, x - mu, then divide by the standard deviation, delta. This results in a distribution with a mean of zero, and a variance and SD of 1. This is less sensitive to outliers, compared to Min-Max scaling but does not offer the strict boundaries that would be helpful. For example, neural networks often expect values between 0 and 1.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler ## the names of the scalers

## Transformation Pipelines

Sometimes we want a chain of transformations to occur in a certain order, and stored in a value that can be used multiple times.
This is called a pipeline and sci-kit learn has built in tools to do exactly that.

In [ ]:
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy = "median")),
    ("attribs_adder", CominedAttributesAdder()),
    ("std_scaler", StandardScaler())
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

So far we have been processing the numerical and categorical columns separately, but this may not be the most concise way to do it. Scikit-Learn has a module called `ColummnTransformer`, where we can specify how we want each column to be transformed. This allows us to use our `num_pipeline` on the numerical columns and modify the categorical columns at the same time.

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ## name of step, the function to apply, the columns to apply on
    ("num", num_pipeline, num_attribs),
    ## name, function, target(s)
    ("cat", OneHotEncoder(), cat_attribs)
])


housing_prepared = full_pipeline.fit_transform(housing)

# Training and Selecting a Model

In [ ]:
import joblib
from sklearn.metrics import root_mean_squared_error

def train_model(model, model_name):
    
    model.fit(housing_prepared, housing_labels)
    predictions = model.predict(housing_prepared)

    joblib.dump(model, "models/" + model_name + ".pkl")
    
    rmse = root_mean_squared_error(housing_labels, predictions)
    # tree_rmse = np.sqrt(tree_mse)
    return f"The Root Mean Square Error is ${rmse:,.2f} ({rmse * 100 / np.median(housing_labels):.2f}%) when the median price is ${np.median(housing_labels):,.2f}"

## Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

train_model(lin_reg, "linear_regression")

## Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()

train_model(tree_reg, "decision_tree_regression")

## Support Vector Machine

In [ ]:
from sklearn.svm import SVR

svm_reg = SVR(kernel = "linear", gamma = "auto", C = 1, max_iter = 5000)
train_model(svm_reg, "support_vector_machine_regression")

## Bayesian Regression

In [ ]:
from sklearn.linear_model import BayesianRidge

br_reg = BayesianRidge(max_iter = 5000)

train_model(br_reg, "Bayesian_ridge_regression")

## Gaussian Process Regression

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor

gp_reg = GaussianProcessRegressor(n_restarts_optimizer = 0, normalize_y = False, random_state = 0)

train_model(gp_reg, "Gaussian_Process_regression")
## this performs much better when the labels are also scaled - worth looking into

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor()

train_model(rf_reg, "Random_Forest_regression")

## Cross-Validation

Now we are at an interesting point, the linear regression was too simple and is underfitting our data but the decision tree was overfitting our data (naturally). We have a few options, we could make our linear model more complex, maybe transform some of the features, introduce complexity with polynomials or collect more data. We could also try to constrain our tree, limit the number of branches it can have and look at the main branches to see how it is decising between the groups. 

First, let's explore the decision tree in more detail. We could split our training set again, using the `train_test_split()` or use cross-validation to get an idea how well it can generalise to new data. Cross-validation can inform us how well our model is going (mean) and how stable or reliable that estimation is (standard deviation). 

In [ ]:
def display_scores(score):
    rmse = np.sqrt(-score)
    print(rmse)
    print(f"Mean: ${rmse.mean():,.2f}")
    print(f"SD: ${rmse.std():,.2f}")

In [ ]:
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_validate
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.filterwarnings('ignore', category = ConvergenceWarning)

## CV expects a utility function instead of a cost function, so larger values are better so make the mean_squared_error negative.

In [ ]:
tree_scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10)
display_scores(tree_scores)

In [ ]:
# rf_scores = cross_val_score(rf_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10)
# display_scores(rf_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10)
display_scores(lin_scores)

In [ ]:
metrics = {
    "mean_squared" : "neg_mean_squared_error", 
    "mean_abs" : "neg_mean_absolute_error", 
    "root_mean_squared" : "neg_root_mean_squared_error"
}

cv_test = cross_validate(
    estimator = lin_reg, 
    X = housing_prepared, 
    y = housing_labels,
    scoring = metrics, 
    cv = 3
)

In [ ]:
cv_test

In [ ]:
# svm_scores = cross_val_score(svm_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10)
# display_scores(svm_scores)

In [ ]:
# br_scores = cross_val_score(br_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10)
# display_scores(br_scores)

In [ ]:
# from joblib import parallel_backend
# with parallel_backend("threading", n_jobs = 2):
    # gp_scores = cross_val_score(gp_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10)
# display_scores(gp_scores)
## too compuationally expensive and not great - don't run

## Fine Tuning Hyperparameters

Let's take the random forest model, since it is one of the best models and is not too computationally expensive.
It has a few hyperparameters we might want to checkout:
- `n_estimators` = number of trees in the forest - default is 100
- `max_depth` = the maximum depth of the tree - default is None
- `min_samples_split`= Minumum number of samples in a node, if less then it doesn't split
- `max_features` = the number of features to look at when splitting 

### Grid Search Cross-Validation

In [ ]:
from sklearn.model_selection import GridSearchCV

## param~_grid should be a dictionary or a list of dictionaries
param_grid = [
    {"n_estimators" : [10, 30, 60], "max_features" : [6, 8], "max_depth" : [5, 10]},
    {"bootstrap" : [True, False], "n_estimators" : [30], "max_features" : [8]}
]

forest_reg = RandomForestRegressor()
grid_cv = GridSearchCV(forest_reg, param_grid, cv = 5, scoring = "neg_mean_squared_error", return_train_score = True)

grid_cv.fit(housing_prepared, housing_labels)

In [82]:
params_test = {"min_samples_split" : [2, 4, 8, 10, 12]}

grid_cv_test = GridSearchCV(tree_reg, params_test, scoring = metrics, cv = 2, refit = False)
grid_cv_test.fit(housing_prepared, housing_labels)

GridSearchCV(cv=2, estimator=DecisionTreeRegressor(),
             param_grid={'min_samples_split': [2, 4, 8, 10, 12]}, refit=False,
             scoring={'mean_abs': 'neg_mean_absolute_error',
                      'mean_squared': 'neg_mean_squared_error',
                      'root_mean_squared': 'neg_root_mean_squared_error'})

In [83]:
# grid_cv_test.cv_results_
grid_cv_test.cv_results_


{'mean_fit_time': array([0.12226272, 0.10044372, 0.08889389, 0.09763849, 0.08416033]),
 'std_fit_time': array([0.01756859, 0.00360096, 0.00557327, 0.01257479, 0.00252581]),
 'mean_score_time': array([0.00350785, 0.00099969, 0.        , 0.        , 0.00175691]),
 'std_score_time': array([0.00150347, 0.00099969, 0.        , 0.        , 0.00024867]),
 'param_min_samples_split': masked_array(data=[2, 4, 8, 10, 12],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_split': 2},
  {'min_samples_split': 4},
  {'min_samples_split': 8},
  {'min_samples_split': 10},
  {'min_samples_split': 12}],
 'split0_test_mean_squared': array([-5.74593901e+09, -5.49940886e+09, -5.16729487e+09, -4.94114451e+09,
        -4.80114194e+09]),
 'split1_test_mean_squared': array([-5.37067988e+09, -5.29016683e+09, -4.98795575e+09, -4.80675514e+09,
        -4.73857318e+09]),
 'mean_test_mean_squared': array([-5.55830944e+09, -5.39478784

In [ ]:
grid_cv.best_params_

In [ ]:
grid_cv.best_estimator_

In [ ]:
np.sqrt(-grid_cv.best_score_) ## $49,586.02

In [ ]:
# grid_cv.cv_results_ get all the results out

In [ ]:
cv_res = grid_cv.cv_results_
for mean_score, params in zip(cv_res["mean_test_score"], cv_res["params"]):
    print(f"${np.sqrt(-mean_score):,.2f} - params: {params}")

### Randomised Search Cross-Validation

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_grid = {
    "n_estimators" : randint(10, 100), 
     "max_features" : randint(3, 17), 
     "max_depth" : randint(3, 20),
    "bootstrap" : [True, False]
    }

forest_reg = RandomForestRegressor()
randgrid_cv = RandomizedSearchCV(forest_reg, 
                                 param_grid, 
                                 cv = 5, 
                                 scoring = "neg_mean_squared_error", 
                                 return_train_score = True,
                                 n_iter = 25, 
                                 n_jobs = 5)

randgrid_cv.fit(housing_prepared, housing_labels)

In [ ]:
randgrid_cv.best_params_

In [ ]:
randgrid_cv.best_estimator_

In [ ]:
np.sqrt(-randgrid_cv.best_score_) ## $49,890.96

In [ ]:
randcv_res = randgrid_cv.cv_results_
for mean_score, sd_score, params in zip(randcv_res["mean_test_score"], randcv_res["std_test_score"], randcv_res["params"]):
    print(f"${np.sqrt(-mean_score):,.2f} (${np.sqrt(sd_score):,.2f}) - params: {params}")

### Exploring the Best Model

In [ ]:
extra_attrs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_hhold"]
cat_encoder = full_pipeline.named_transformers_.cat.categories_[0]
cat_attr = list(cat_encoder)
attrs = num_attribs + extra_attrs + cat_attr

## if we still had a dataframe instead of a list of numpy arrays, we could just use this to get the feature names
# list(housing_prepared)

sorted(zip(randgrid_cv.best_estimator_.feature_importances_, attrs), reverse = True)

## Testing with Log Transformation

In [ ]:
from sklearn.model_selection import train_test_split

housing2 = pd.read_csv("../datasets/housing/housing.csv")

## get a list of the columns
cols = ["total_rooms", "total_bedrooms", "population", "households", "median_income"]

for col in cols:
    housing2[col] = np.log1p(housing2[col])

housing2.head()

In [ ]:
%matplotlib inline
## Let's visualise the distributions of the numeric variables
housing2.hist(bins = 50, figsize = (20, 15))
plt.show()

In [ ]:
X = housing2.drop("median_house_value", axis = 1)
y = housing2["median_house_value"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2 , random_state = 42)

In [ ]:
X_train["ocean_proximity"].value_counts()

num_attribs = list(X_train)
cat_attribs = [num_attribs.pop(8)]  ## this remove ocean_proximity from the num_atribs

## reminder of the full pipeline
full_pipeline = ColumnTransformer([
    ## name of step, the function to apply, the columns to apply on
    ("num", num_pipeline, num_attribs),
    ## name, function, target(s)
    ("cat", OneHotEncoder(), cat_attribs)
])

housing_prepared2 = full_pipeline.fit_transform(X_train)

## get the names for all the columns, including the new and formatted ones.
all_columns = num_attribs + [
    "rooms_per_household", "population_per_household", "bedrooms_per_room"] + [
    "<1H OCEAN", "INLAND", "ISLAND", "NEAR BAY", "NEAR OCEAN"]

housing_prepared2 = pd.DataFrame(housing_prepared2, columns = all_columns)
housing_prepared2